In [1]:
from pymilvus import MilvusClient
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pickle
import tqdm
import pandas as pd
import requests
import torch
import numpy as np

#########   KEY IN YOUR IP  #########
client = MilvusClient(
    uri="http://192.168.1.111:19530"
)

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client.drop_partition(collection_name='image', partition_name='1_0')

In [20]:
import pickle
with open('D:/NTU/CSIE/112/DBMS/github/pretrain_database/testing/test_cluster_info.pkl', 'rb') as file:
    save_pairs = pickle.load(file)
save_pairs[1]['subgroup'].keys()

dict_keys([0, 1, 2])

In [6]:
res = client.query(
        collection_name="image",
        filter="",
        partition_names=['0_13'],
        output_fields=["count(*)"]
    )
size = res[0]["count(*)"]
print(size)

res = client.list_partitions(collection_name='image')
print(res)

RPC error: [query], <MilvusException: (code=201, message=fail to Query on QueryNode 7: partition not loaded[partition=[449961400704324806]])>, <Time:{'RPC start': '2024-05-25 23:05:02.864670', 'RPC error': '2024-05-25 23:05:03.472249'}>
Failed to query collection: image


MilvusException: <MilvusException: (code=201, message=fail to Query on QueryNode 7: partition not loaded[partition=[449961400704324806]])>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
def get_clip_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model.get_image_features(**inputs)
    outputs = outputs.cpu()  
    return outputs

In [ ]:
image_path = input('Paste the path of your image here:')
image_embedding = get_clip_embedding(image_path)
image_embedding = image_embedding.detach().numpy()
image_embedding = image_embedding.squeeze()
image_embedding = np.array(image_embedding)

In [ ]:
data = [image_embedding]
res = client.search(
    collection_name="image", # Replace with the actual name of your collection
    data=data,
    limit=1,  # Max. number of search results to return
    search_params={"metric_type": "COSINE", "params": {}}, # Search parameters
    partition_names=["_default"] # Partition names to search in
)

id = int(input('Key in id:'))
xxx = input('Key in name:')

partition_id = res[0][0]['id']
print(partition_id)
data = {"id": id, "vector": image_embedding, "name": xxx}
res = client.insert(
    collection_name="image",
    data=data,
    partition_name="partition"+str(partition_id)
)